# ToyMD

## How to run the exercises
Access the NVE version of the code on Moodle and unzip in the Ex5 folder in your noto repository. 

In [2]:
import nglview as nv
import pytraj as pt
import matplotlib.pyplot as plt

In [3]:
import sys
sys.path.append("..")
import helpers
helpers.set_style()

In [4]:
import numpy as np

# Exercises

## MD Initialization and Temperature

<div class="exercise admonition" name="5ex1" style="padding: 10px">
<p class="title">Exercise 1</p>
Implement the initialization described in the Theory section in our ToyMD code in `toy_md.py` in the section `##INITIALIZATION HERE##`. For this you should pick random velocities using `r.random.standard_normal((dim1,dim2))` which will produce random numbers between `0` and `1`. You need to shift this random gaussian distribution appropriately and then multiply it such that the width of the velocity distribution matches the kinetic energy at the target temperature. Use the variables `masses[i]` for the mass of particle `i` and Boltzmanns constant `0.00831415`. Remember that for each degree of freedom (e.g velocity in x direction) {eq}`equipart_x` holds.
</div>

<div class="exercise admonition" name="5ex2" style="padding: 10px">
<p class="title">Exercise 2</p>
Implement the Berendsen thermostat in the `toy_md.py` and `toy_md_integrate.py` (change the `compute_lambda_T` function) files. 
</div>

<div class="exercise admonition" name="5ex3" style="padding: 10px">
<p class="title">Exercise 3</p>
A better thermostat is the Andersen thermostat. It can be implemented as follows. Describe what problems this thermostat will present to us e.g for sampling of diffusion coefficients. What advantage does this thermostat have compared to the Berendsen thermostat?
</div>

``` python
#Andersen Thermostat
for i in range(N):
    if (r.random()<float(md_params["tau-T"])*float(md_params['time-step'])):# pick random particles according to nu
        sd=np.sqrt(0.00831415*float(md_params['time-step'])/masses[i])  # Velocity distribution at target temperature
        velocities[i]=0+np.random.randn(1,3)*sd # set new random velocities
```

## Force field


Now we can run a MD simulation in our desired NVT ensemble using our code on simple systems. Let's simulate a box of CO2 molecules. <br>
Investigate the `force_field.txt` file in the `carbon-dioxide` folder and then run a short molecular dynamics simulation using the following parameters:

```
number-of-steps  2000 # Number of integration time steps
time-step        0.001  # Integration time step (picoseconds)
temperature      50  # Simulation temperature
tau-T            0.05 # Temperature coupling time (picoseconds)
output-frequency  10   # Store coordinates every N steps

```

The simulation is run using 

    path/to/toy_md.py -c co2.pdb -p params.txt -f force_field.txt -o co2-traj.pdb -w co2-final.pdb

where `co2-traj.pdb` is a trajectory written each `output-frequency` steps and `co2-final.pdb` is the final geometry after the simulation has run the specified number of steps. 

<div class="exercise admonition" name="5ex4" style="padding: 10px">
<p class="title">Exercise 4</p>
Visualize the trajectory and visualize the distance of a C-O bond. Explain the flucations that you observe. What does the average value correspond to?
</div>

### Visualization of the trajectory

Here you can visualize the trajectory to see if your simulation is running correctly. A quick visualization often highlights if the parameters/simulation are correct.

In [5]:
# ADJUST THE PATHS AS NECESSARY TO MATCH THE NAME OF YOUR FOLDERS
traj = pt.load('./ToyMD-master/carbon-dioxide/co2-traj.pdb',  top='./ToyMD-master/carbon-dioxide/co2.pdb')
view = nv.show_pytraj(traj)
view.clear_representations()
view.add_representation('ball+stick')
view.center()
view

NGLWidget(max_frame=49)

In [6]:
# lets look at which atoms are in our trajectory
print([atom.name for atom in traj.top.atoms][:20])

['C', 'O1', 'O2', 'C', 'O1', 'O2', 'C', 'O1', 'O2', 'C', 'O1', 'O2', 'C', 'O1', 'O2', 'C', 'O1', 'O2', 'C', 'O1']


In [ ]:
#@1 is the first atom, @2 is the second atom etc. 
# We want to compute the C - O1 and C - O2 bond using pytraj
distance1 = pt.distance(traj, "SELECTOION HERE") # modify here
distance2 = pt.distance(traj, "SELECTION HERE")# modify here
forcefield = 0  #check force_field.txt in ToyMD folder for this value # modify here

In [ ]:
fig, ax =plt.subplots(1)

ax.plot(distance1, label="C-O1")
ax.plot(distance2, label="C-O2")
ax.axhline(forcefield, label="distance from forcefield")
ax.set_xlabel("timestep")
ax.set_ylabel(r'distance ($\AA$)')
ax.set_title('Distance')
ax.legend()

plt.show()

<div class="exercise admonition" name="5ex5" style="padding: 10px">
<p class="title">Exercise 5</p>
Visualize the radial distribution function of this system using below code. What do you observe?
</div>

### Compute RDF

We can use the `pytraj` library to compute the radial distribution function of a sample trajectory that we obtain from the example files of pytraj. 

(There is currently a bug with PDB trajectories so we use the below example trajectory that contains a small protein and some water molecules). 

In [9]:
traj = pt.datafiles.load_tz2_ortho()

In [10]:
view = nv.show_pytraj(traj)
view.clear_representations()
view.add_representation('ball+stick')
view.center()
view

NGLWidget(max_frame=9)

In [ ]:
# select residues using :RES@atomname e.g :WAT@O, :SER@CA
# select residue ids using :1-2@
rdf_data = pt.rdf(traj, solvent_mask=':WAT@O', bin_spacing=0.5, maximum=10.0, solute_mask='MODIFY HERE')

In [ ]:
fig, ax =plt.subplots(1)

ax.plot(rdf_data[0], rdf_data[1])
ax.set_xlabel(r"r ( $\AA$ )")
ax.set_ylabel('g(r)')
ax.set_title('RDF')

plt.show()

<div class="exercise admonition" name="5bex1" style="padding: 10px">
<p class="title">Bonus Exercise 1</p>
How would the radial distribution function look like for a heterogenous selection?  You can modify the below code and use a different solute mask (e.g :1@CA) to look at the RDF of water at the N terminus of the protein. 
</div>

## Timestep and coupling

<div class="exercise admonition" name="5ex6" style="padding: 10px">
<p class="title">Exercise 6</p>
Test the influence of the coupling parameter $\tau$ and timestep $dt$. 
Test the combinations of:
- large $\tau$ and small $dt$
- large $dt$ and small $\tau$ 
- very large $dt$ and $\tau$ 
</div>